<a href="https://colab.research.google.com/github/pan-aleksij/MEPHI_/blob/Basics_python/%D0%9A%D0%B0%D0%B1%D0%B0%D0%BD%D0%BE%D0%B2_%D0%90_%D0%AD_%D0%BE%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_py_3_%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U kaggle_environments

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 7.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

/usr/local/lib/python3.10/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-pa

Опишем поведение агента, всегда играющего "камень" - это значение 0

In [ ]:
%%writefile rock_agent.py

#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 0

Writing rock_agent.py


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Попробуем теперь использовать информацию о прошлых действиях противника. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [ ]:
%%writefile copy_opponent.py

#Example
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Writing copy_opponent.py


Создаю агентов

In [ ]:
# Создаем агентов
#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
import random
# Выбирает только камень
def rock_agent(observation, configuration):
    return 0

# Выбирает только бумагу
def paper_agent(observation, configuration):
    return 1

# Выбирает только ножницы
def scissors_agent(observation, configuration):
    return 2

# Выбирает случаянный результат
def random_agent(observation, configuration):
    return random.choice([0, 1, 2])

# Повторяет последний результат оппонента
def copy_opponent_agent(observation, configuration):
    if observation.get('step', 0) > 0:
        return observation.get('lastOpponentAction', 0)
    else:
        return random.choice([0, 1, 2])

# Повторяет послений результат и бъет его
def beat_last_agent(observation, configuration):
    if observation.get('step', 0) > 0:
        return (observation.get('lastOpponentAction', 0) + 1) % 3
    else:
        return random.choice([0, 1, 2])

# Смешанная стратегия с уклоном в камень
def mixed_strategy_agent_rock(observation, configuration):
    mixed_choices = [0, 0, 0, 1, 1, 2]
    return random.choice(mixed_choices)

# Смешанная стратегия с уклоном в бумагу
def mixed_strategy_agent_paper(observation, configuration):
    mixed_choices = [0, 0, 1, 1, 1, 2, 2]
    return random.choice(mixed_choices)

# Смешанная стратегия с уклоном в ножницы
def mixed_strategy_agent_scissors(observation, configuration):
    mixed_choices = [0, 0, 1, 1, 2, 2, 2, 2]
    return random.choice(mixed_choices)

# Агент, который циклически выбирает ходы (камень, бумага, ножницы).
def cycle_agent(observation, configuration):
    if observation.get('step', 0) == 0:
        return 0
    else:
        return (observation.get('lastOpponentAction', 0) + 1) % 3

# Агент, который выбирает ход, который бьет наиболее часто используемый противником ход.
def beat_frequency_agent(observation, configuration):
    if observation.get('step', 0) > 0:
        counts = [0, 0, 0]
        for action in observation.get('history', []):
            counts[action] += 1
        return (counts.index(max(counts)) + 1) % 3
    else:
        return random.choice([0, 1, 2])

# Вбирает только ножницы или камень
def only_rock_scissors(observation, configuration):
    mixed_choices = [0, 0, 2, 2, 2, 2]
    return random.choice(mixed_choices)

# Если оппонент использует повторяющийся цикл, то он его будет бить
def anti_cycle_agent(observation, configuration):
    if observation.get('step', 0) > 0:
        return (observation.get('lastOpponentAction', 0) + 2) % 3
    else:
        return random.choice([0, 1, 2])

Объявляю список агентов

In [ ]:
# Список агентов
agents = [
    rock_agent,
    paper_agent,
    scissors_agent,
    random_agent,
    copy_opponent_agent,
    beat_last_agent,
    mixed_strategy_agent_rock,
    mixed_strategy_agent_paper,
    mixed_strategy_agent_scissors,
    cycle_agent,
    beat_frequency_agent,
    only_rock_scissors,
    anti_cycle_agent
]

Запуск турнира и вывод информации о них

In [ ]:

from kaggle_environments import make, evaluate
from collections import defaultdict

# Запуск турнира
results = []
for i in range(len(agents)):
    for j in range(i + 1, len(agents)):
        agent1 = agents[i]
        agent2 = agents[j]
        match_result = evaluate(
            "rps",  # environment to use
            [agent1, agent2],  # agents to evaluate
            configuration={"episodeSteps": 100}  # number of episodes
        )
        results.append((agent1.__name__, agent2.__name__, match_result))

# Вывод результатов
print(f'Результаты турниров:')
for result in results:
    print(f"Agent 1: {result[0]}, Agent 2: {result[1]}, Result: {result[2]}")
print('-' * 40)

Результаты турниров:
Agent 1: rock_agent, Agent 2: paper_agent, Result: [[-99.0, 99.0]]
Agent 1: rock_agent, Agent 2: scissors_agent, Result: [[99.0, -99.0]]
Agent 1: rock_agent, Agent 2: random_agent, Result: [[0, 0]]
Agent 1: rock_agent, Agent 2: copy_opponent_agent, Result: [[0, 0]]
Agent 1: rock_agent, Agent 2: beat_last_agent, Result: [[-98.0, 98.0]]
Agent 1: rock_agent, Agent 2: mixed_strategy_agent_rock, Result: [[0, 0]]
Agent 1: rock_agent, Agent 2: mixed_strategy_agent_paper, Result: [[0, 0]]
Agent 1: rock_agent, Agent 2: mixed_strategy_agent_scissors, Result: [[43.0, -43.0]]
Agent 1: rock_agent, Agent 2: cycle_agent, Result: [[-98.0, 98.0]]
Agent 1: rock_agent, Agent 2: beat_frequency_agent, Result: [[-99.0, 99.0]]
Agent 1: rock_agent, Agent 2: only_rock_scissors, Result: [[62.0, -62.0]]
Agent 1: rock_agent, Agent 2: anti_cycle_agent, Result: [[98.0, -98.0]]
Agent 1: paper_agent, Agent 2: scissors_agent, Result: [[-99.0, 99.0]]
Agent 1: paper_agent, Agent 2: random_agent, Res

In [ ]:
# Подсчет результатов
agent_scores = defaultdict(lambda: {"wins": 0, "losses": 0, "draws": 0})

for result in results:
    agent1_name = result[0]
    agent2_name = result[1]
    match_result = result[2][0]

    if match_result[0] > match_result[1]:
        agent_scores[agent1_name]["wins"] += 1
        agent_scores[agent2_name]["losses"] += 1
    elif match_result[0] < match_result[1]:
        agent_scores[agent1_name]["losses"] += 1
        agent_scores[agent2_name]["wins"] += 1
    else:
        agent_scores[agent1_name]["draws"] += 1
        agent_scores[agent2_name]["draws"] += 1

# Вывод результатов
print(f'Результаты матчей:')
for agent, scores in agent_scores.items():
    print(f"Agent: {agent}, Wins: {scores['wins']}, Losses: {scores['losses']}, Draws: {scores['draws']}")
print('-' * 40)

Результаты матчей:
Agent: rock_agent, Wins: 4, Losses: 4, Draws: 4
Agent: paper_agent, Wins: 3, Losses: 5, Draws: 4
Agent: scissors_agent, Wins: 3, Losses: 5, Draws: 4
Agent: random_agent, Wins: 0, Losses: 0, Draws: 12
Agent: copy_opponent_agent, Wins: 2, Losses: 1, Draws: 9
Agent: beat_last_agent, Wins: 5, Losses: 2, Draws: 5
Agent: mixed_strategy_agent_rock, Wins: 1, Losses: 2, Draws: 9
Agent: mixed_strategy_agent_paper, Wins: 0, Losses: 0, Draws: 12
Agent: mixed_strategy_agent_scissors, Wins: 2, Losses: 1, Draws: 9
Agent: cycle_agent, Wins: 7, Losses: 0, Draws: 5
Agent: beat_frequency_agent, Wins: 3, Losses: 5, Draws: 4
Agent: only_rock_scissors, Wins: 4, Losses: 3, Draws: 5
Agent: anti_cycle_agent, Wins: 1, Losses: 7, Draws: 4
----------------------------------------
